<h3> Data Cleaning </h3>

In [228]:
import numpy as np
import pandas as pd
import fuzzywuzzy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [229]:
delegates = pd.read_csv("Data/Delegates/cleaned/constitutional_convention_delegates_cleaned.csv").drop('Unnamed: 0', axis = 1)
state_delegates = pd.read_csv("Data/Delegates/cleaned/State_Delegates_cleaned.csv").drop('Unnamed: 0', axis = 1)

In [230]:
final_matching = pd.read_csv('Data/final_matching.csv').drop(['Unnamed: 0', 'Scores'], axis = 1)

In [231]:
spec_names = final_matching[final_matching['Delegates'] != final_matching['Loan Matches']]

# Delegate Asset Database

In [232]:
delegates_assets = delegates[['full name 1', 'full name 2', 'sign']]
state_delegate_assets = state_delegates[['full name 1', 'State', 'Vote']]
state_delegate_assets.rename(columns = {"Vote":"sign", "State":"delegate_state"}, inplace = True)
total_assets = pd.concat([delegates_assets, state_delegate_assets]).reset_index(drop = True)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


In [233]:
#amending existing cases

In [234]:
total_assets['full name 2'] = total_assets['full name 1']
total_assets_full = total_assets

In [235]:
merge_1 = pd.merge(total_assets_full.drop('full name 2', axis = 1), 
                   spec_names, left_on = 'full name 1', 
                   right_on = 'Delegates').drop('full name 1', axis = 1)
merge_2 = pd.merge(total_assets_full.drop('full name 1', axis = 1), 
                   spec_names, left_on = 'full name 2', 
                   right_on = 'Delegates').drop('full name 2', axis = 1)
merged = pd.concat([merge_1, merge_2]).drop_duplicates()

In [236]:
total_assets_full.columns = ['delegate_state', 'delegate name', 'match name', 'sign']
merged.columns =  ['delegate_state', 'sign',  'delegate name', 'match name']

In [237]:
total_assets_final = pd.concat([total_assets_full, merged])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


### Full Name - First/Last Name Mapping

In [238]:
state_firstlast = state_delegates[['full name 1', 'First Name', 'Last Name']].rename(columns = {'full name 1': 'Full Name'})
const_firstlast = delegates[['full name 1', 'first name', 'last name']].rename(columns = {'first name':'First Name',
                                                                                          'last name':'Last Name',
                                                                                          'full name 1': 'Full Name'})
cum_firstlast = pd.concat([state_firstlast, const_firstlast])

# Loans Dataset Matching

In [239]:
#prepare loan dataset
loans = pd.read_csv("Data/Pre1790/cleaned/loan_office_certificates_9_states_cleaned.csv").drop('Unnamed: 0', axis = 1)

loans = loans.rename(columns = {"First Name 1 ":"First Name 1",
                                "Last Name 1 ":"Last Name 1",
                                "Specie Value ":"Specie Value"})

loans['full name 1'] = loans['First Name 1'] + " " + loans['Last Name 1']
loans['full name 2'] = loans['First Name 2'] + " " + loans['Last Name 2']
loans['full name 3'] = loans['First Name 3'] + " " + loans['Last Name 3']

In [240]:
#specie, face and num

In [241]:
loans['index'] = loans.index

loan_small_f1 = loans[['State', 'Year', 'Month', 'Day', 'full name 1', 'Face Value','Specie Value', 'notes', 'original text', 'index']]
loan_small_f2 = loans[['State', 'Year', 'Month', 'Day', 'full name 2', 'Face Value','Specie Value', 'notes', 'original text', 'index']]
loan_small_f3 = loans[['State', 'Year', 'Month', 'Day', 'full name 3', 'Face Value','Specie Value', 'notes', 'original text', 'index']]
loan_office_cert_assets = pd.merge(total_assets_final,
                                   loan_small_f1[loan_small_f1['full name 1'].apply(lambda x: not pd.isnull(x))],
                                   left_on = 'match name', right_on = 'full name 1').drop('full name 1', axis = 1)
add_to_assets = pd.merge(total_assets_final,
                         loan_small_f2[loan_small_f2['full name 2'].apply(lambda x: not pd.isnull(x))],
                         left_on = 'match name', right_on = 'full name 2').drop('full name 2', axis = 1)
loan_office_cert_assets = pd.concat([loan_office_cert_assets, add_to_assets])
add_to_assets = pd.merge(total_assets_final,
                         loan_small_f3[loan_small_f3['full name 3'].apply(lambda x: not pd.isnull(x))],
                         left_on = 'match name', right_on = 'full name 3').drop('full name 3', axis = 1)
loan_office_cert_assets = pd.concat([loan_office_cert_assets, add_to_assets])
loan_office_cert_assets = loan_office_cert_assets.drop_duplicates(['delegate name', 'index'])
loan_office_cert_assets['delegate_state'] = loan_office_cert_assets['delegate_state'].apply(lambda x: 'Constitution' if pd.isnull(x) else x)

In [242]:
loan_office_cert_assets.rename(columns={'Face Value':'Loan Office Certificates Face Value','Specie Value':'Loan Office Certificates Specie Value'}, inplace = True)

In [243]:
loan_office_cert_assets = pd.merge(cum_firstlast, loan_office_cert_assets, left_on = 'Full Name', right_on = 'delegate name').drop('Full Name', axis = 1).drop_duplicates()
loan_office_cert_assets_agg = loan_office_cert_assets.groupby(['delegate name', 'First Name', 'Last Name', 'delegate_state', 'sign'])[['Loan Office Certificates Face Value', 'Loan Office Certificates Specie Value']].agg(['sum','count'])
loan_office_cert_assets_agg

Loan Office Certificates Face Value  \
                                                                                          sum   
delegate name    First Name Last Name delegate_state sign                                       
Aaron Austin     Aaron      Austin    CT             yes                                 3700   
Aaron Wood       Aaron      Wood      MA             no                                   600   
Abraham Faw      Abraham    Faw       MD             yes                                  400   
Abraham Holmes   Abraham    Holmes    MA             no                                 13200   
Abraham Smith    Abraham    Smith     MA             no                                   300   
...                                                                                       ...   
William White    William    White     VA             no                                 11400   
William Williams William    Williams  CT             yes                                15600   
William Wilson   William    Wilson    PA             yes                                 2800   
William Woodhull William    Woodhull  NJ             yes                                 1000   
Williams John    Williams   John      RI             no                                107800   

                                                                 \
                                                          count   
delegate name    First Name Last Name delegate_state sign         
Aaron Austin     Aaron      Austin    CT             yes      3   
Aaron Wood       Aaron      Wood      MA             no       1   
Abraham Faw      Abraham    Faw       MD             yes      1   
Abraham Holmes   Abraham    Holmes    MA             no       8   
Abraham Smith    Abraham    Smith     MA             no       1   
...                                                         ...   
William White    William    White     VA             no      18   
William Williams William    Williams  CT             yes     28   
William Wilson   William    Wilson    PA             yes      9   
William Woodhull William    Woodhull  NJ             yes      2   
Williams John    Williams   John      RI             no     176   

                                                          Loan Office Certificates Specie Value  \
                                                                                            sum   
delegate name    First Name Last Name delegate_state sign                                         
Aaron Austin     Aaron      Austin    CT             yes                              180.06667   
Aaron Wood       Aaron      Wood      MA             no                                66.72500   
Abraham Faw      Abraham    Faw       MD             yes                               35.53889   
Abraham Holmes   Abraham    Holmes    MA             no                               527.52500   
Abraham Smith    Abraham    Smith     MA             no                               123.49170   
...                                                                                         ...   
William White    William    White     VA             no                              1197.58450   
William Williams William    Williams  CT             yes                             3957.08889   
William Wilson   William    Wilson    PA             yes                              272.00277   
William Woodhull William    Woodhull  NJ             yes                               51.06944   
Williams John    Williams   John      RI             no                              9740.07502   

                                                                 
                                                          count  
delegate name    First Name Last Name delegate_state sign        
Aaron Austin     Aaron      Austin    CT             yes      3  
Aaron Wood       Aaron      Wood      MA             no       1  
Abraham Faw      Abraham    Faw       MD   

# Pierce Certificates

In [244]:
#prepare pierce certificates
pierce = pd.read_excel("Data/Pre1790/Pierce_Certs_cleaned_2019.xlsx")

#clean first and last names
pierce = pierce[pierce['First'].apply(lambda x: not pd.isnull(x))]
pierce_ind = pierce[pierce['Last'].apply(lambda x: type(x) != str)].index
#fix issue with types for names
pierce.loc[pierce_ind, 'Last'] = ['True'] * len(pierce_ind)
#create full name
pierce['full name 1'] = pierce['First'] + " " + pierce['Last']

In [245]:
pierce_small_f1 = pierce[['State','full name 1', 'Value', 'CN']]
pierce_cert_assets = pd.merge(total_assets_final,
                              pierce_small_f1[pierce_small_f1['full name 1'].apply(lambda x: not pd.isnull(x))],
                              left_on = 'match name', right_on = 'full name 1').drop('full name 1', axis = 1)
pierce_cert_assets = pierce_cert_assets.drop_duplicates(['delegate name', 'CN'])
pierce_cert_assets['delegate_state'] = pierce_cert_assets['delegate_state'].apply(lambda x: 'Constitution' if pd.isnull(x) else x)

In [246]:
pierce_cert_assets.rename(columns={'Value':'Pierce Certificates Value'}, inplace = True)

In [247]:
pierce_cert_assets = pd.merge(cum_firstlast, pierce_cert_assets, left_on = 'Full Name', right_on = 'delegate name').drop('Full Name', axis = 1).drop_duplicates()
pierce_cert_assets_agg = pierce_cert_assets.groupby(['delegate name', 'First Name', 'Last Name', 'delegate_state', 'sign'])[['Pierce Certificates Value']].agg(['sum','count'])
pierce_cert_assets_agg

Pierce Certificates Value  \
                                                                                  sum   
delegate name      First Name Last Name delegate_state sign                             
A. Fisher          A.         Fisher    MA             yes                     199.10   
Aaron Hall         Aaron      Hall      NH             yes                      67.20   
Aaron Wood         Aaron      Wood      MA             no                      120.60   
Abraham Baldwin    Abraham    Baldwin   Constitution   yes                    3746.75   
Abraham Smith      Abraham    Smith     MA             no                      290.20   
...                                                                               ...   
William White      William    White     VA             no                     4272.54   
William Williams   William    Williams  CT             yes                    4062.61   
William Wilson     William    Wilson    PA             yes                    4800.33   
Williams John      Williams   John      RI             no                     6900.62   
Zachariah Johnston Zachariah  Johnston  VA             yes                     147.20   

                                                                   
                                                            count  
delegate name      First Name Last Name delegate_state sign        
A. Fisher          A.         Fisher    MA             yes      2  
Aaron Hall         Aaron      Hall      NH             yes      2  
Aaron Wood         Aaron      Wood      MA             no       2  
Abraham Baldwin    Abraham    Baldwin   Constitution   yes      6  
Abraham Smith      Abraham    Smith     MA             no       5  
...                                                           ...  
William White      William    White     VA             no      26  
William Williams   William    Williams  CT             yes     49  
William Wilson     William    Wilson    PA             yes     36  
Williams John      Williams   John      RI             no      64  
Zachariah Johnston Zachariah  Johnston  VA             yes      1  

[550 rows x 2 columns]

# Marine Certificates

In [248]:
#Read in file
marine = pd.read_excel("Data/Pre1790/Marine_Liquidated_Debt_Certificates.xlsx", header = 11)
cols = marine.columns = ["Page","JPEG number","Number","Letter","Date of the Certificate: Month",
                         "Date of the Certificate: Day","Date of the Certificate: Year","First name","Last name",
                         "Title","Time when the debt became due: Month","Time when the debt became due: Day",
                         "Time when the debt became due: Year","Dollars","90th","Total Dollars_1","Total Dollars_2",
                         "Line Strike Thorugh: Yes?","Line Strike Thorugh: Note","Notes"]
marine.columns = cols

marine = marine[marine['First name'].apply(lambda x: not pd.isnull(x))]

#create full name
marine['full name 1'] = marine['First name'] + " " + marine['Last name']

In [249]:
marine_small_f1 = marine[['full name 1', 'Number', 'Dollars', '90th']]
marine_cert_assets = pd.merge(total_assets_final,
                              marine_small_f1[marine_small_f1['full name 1'].apply(lambda x: not pd.isnull(x))],
                              left_on = 'match name', right_on = 'full name 1').drop('full name 1', axis = 1)
marine_cert_assets = marine_cert_assets.drop_duplicates(['delegate name', 'Number'])
marine_cert_assets['delegate_state'] = marine_cert_assets['delegate_state'].apply(lambda x: 'Constitution' if pd.isnull(x) else x)


In [250]:
marine_cert_assets.rename(columns={'Dollars':'Marine Certificates Dollars','90th':'Marine Certificates 90th'}, inplace = True)

In [251]:
marine_cert_assets = pd.merge(cum_firstlast, marine_cert_assets, left_on = 'Full Name', right_on = 'delegate name').drop('Full Name', axis = 1).drop_duplicates()
marine_cert_assets_agg = marine_cert_assets.groupby(['delegate name', 'First Name', 'Last Name', 'delegate_state', 'sign'])[['Marine Certificates Dollars', 'Marine Certificates 90th']].agg(['sum','count'])
marine_cert_assets_agg

Marine Certificates Dollars  \
                                                                                            sum   
delegate name          First Name     Last Name delegate_state sign                               
Benjamin Browne        Benjamin       Browne    MA             yes                        141.0   
Brown John             Brown          John      RI             yes                        262.0   
Clark John I           Clark          John I    RI             yes                         71.0   
David Bell             David          Bell      VA             no                         108.0   
David Rice             David          Rice      NC             yes                         76.0   
Davis John             Davis          John      RI             yes                         35.0   
George Houser          George         Houser    NC             yes                       1047.0   
James Neal             James          Neal      MA             no                          42.0   
John Brown             John           Brown     NC             no                         262.0   
John Davis             John           Davis     MA             yes                         35.0   
John Dunlap            John           Dunlap    MA             yes                          1.0   
John Elliot            John           Elliot    GA             yes                        194.0   
John Hunter            John           Hunter    SC             yes                        310.0   
John Jones             John           Jones     DE             yes                         77.0   
John K. Smith          John K.        Smith     MA             yes                         23.0   
John Rhea              John           Rhea      NC             yes                         43.0   
John Smith             John           Smith     NY             yes                         23.0   
John Taylor            John           Taylor    MA             no                          72.0   
John Taylor Gilman     John Taylor    Gilman    NH             yes                         72.0   
John Williams          John           Williams  VA             yes                         28.0   
Jonathan Smith         Jonathan       Smith     MA             no                         106.0   
Nathaniel Rogers       Nathaniel      Rogers    NH             yes                         69.0   
Samuel Johnston        Samuel         Johnston  NC             yes                         52.0   
Thomas Jones           Thomas         Jones     MA             yes                         95.0   
William Hart           William        Hart      CT             yes                         45.0   
William Jones          William        Jones     MA             no                        1871.0   
William Morris         William        Morris    MD             yes                        471.0   
William Samuel Johnson William Samuel Johnson   Constitution   yes                         52.0   
William Scott          William        Scott     SC             yes                         84.0   
William Smith          William        Smith     SC             yes                        636.0   
Williams John          Williams       John      RI             no                          28.0   

                                                                           \
                                                                    count   
delegate name          First Name     Last Name delegate_state sign         
Benjamin Browne        Benjamin       Browne    MA             yes      1   
Brown John             Brown          John      RI             yes      1   
Clark John I           Clark          John I    RI             yes      1   
David Bell             David          Bell      VA             no       1   
David Rice             David          Rice      NC             yes      1   
Davis John             Davis          John      RI             yes      1   
George H

# Connecticut

In [252]:
#Read in file
CT = pd.read_excel("Data/Pre1790/liquidated_debt_certificates_CT.xlsx", header = 11)
cols = CT.columns = ["Register Page","JPEG number","Number","Letter","Date of the Certificate: Month",
                     "Date of the Certificate: Day","Date of the Certificate: Year","First name","Last name",
                     "Title","First name 2","Last name 2","Title 2","Time when the debt became due: Month",
                     "Time when the debt became due: Day", "Time when the debt became due: Year","Dollars","90th",
                     "Line Strike Thorugh: Yes?","Line Strike Thorugh: Note","Notes"]
CT.columns = cols
CT = CT[CT['First name'].apply(lambda x: not pd.isnull(x))]

#create full name
CT['full name 1'] = CT['First name'] + " " + CT['Last name']
CT['full name 2'] = CT['First name 2'] + " " + CT['Last name 2']

In [253]:
CT_short = CT[['full name 1', 'full name 2', 'Number', 'Dollars', '90th']]
CT_short['Number'] = CT_short['Number'].apply(lambda x: 'CT_' + str(int(x)))

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


# Delaware

In [254]:
#Read in file
DE = pd.read_excel("Data/Pre1790/liquidated_debt_certificates_DE.xlsx", header = 11)
cols = DE.columns = ["JPEG number","Number","Letter","Date of the Certificate: Month",
                     "Date of the Certificate: Day","Date of the Certificate: Year","Title","First name","Last name",
                     "Time when the debt became due: Month",
                     "Time when the debt became due: Day", "Time when the debt became due: Year","Dollars","90th",
                     "Line Strike Through","Note"]
DE.columns = cols

DE = DE[DE['First name'].apply(lambda x: not pd.isnull(x))]

#create full name
DE['full name 1'] = DE['First name'] + " " + DE['Last name']

In [255]:
DE_short = DE[['full name 1', 'Number', 'Dollars', '90th']]
DE_short['Number'] = DE_short['Number'].apply(lambda x: 'DE_' + str(int(x)))
states_short = pd.concat([DE_short, CT_short])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


# Massachusetts

In [256]:
#Read in file
MA = pd.read_excel("Data/Pre1790/liquidated_debt_certificates_MA.xlsx", header = 11)
cols = ["JPEG number","Page", "Number","Letter","Date of the Certificate: Month",
                     "Date of the Certificate: Day","Date of the Certificate: Year","First name","Last name","Title",
                     "First name 2","Last name 2", "Title 2","Time when the debt became due: Month",
                     "Time when the debt became due: Day", "Time when the debt became due: Year","Dollars","90th",
                     "Line Strike Through","Note"]
MA = MA[MA.columns[0:20]]
MA.columns = cols

MA = MA[MA['First name'].apply(lambda x: not pd.isnull(x))]

#create full name
MA['full name 1'] = MA['First name'] + " " + MA['Last name']
MA['full name 2'] = MA['First name 2'] + " " + MA['Last name 2']

In [257]:
MA_short = MA[['full name 1', 'full name 2', 'Number', 'Dollars', '90th']]
MA_short['Number'] = MA_short['Number'].apply(lambda x: 'MA_' + str(int(x)))
states_short = pd.concat([states_short, MA_short])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


# New Hampshire

In [258]:
#Read in file
NH = pd.read_excel("Data/Pre1790/liquidated_debt_certificates_NH.xlsx", header = 11)
cols = ["Page","JPEG number","Number","Letter","Date of the Certificate: Month",
                         "Date of the Certificate: Day","Date of the Certificate: Year","First name","Last name",
                         "Title","Time when the debt became due: Month","Time when the debt became due: Day",
                         "Time when the debt became due: Year","Dollars","90th",
                         "Line Strike Thorugh: Yes?","Line Strike Thorugh: Note","Notes"]
NH.columns = cols

NH = NH[NH['First name'].apply(lambda x: not pd.isnull(x))]

#create full name
NH['full name 1'] = NH['First name'] + " " + NH['Last name']

In [259]:
NH_short = NH[['full name 1', 'Number', 'Dollars', '90th']]
NH_short['Number'] = NH_short['Number'].apply(lambda x: 'NH_' + str(int(x)))
states_short = pd.concat([states_short, NH_short])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


# New Jersey

In [260]:
#Read in file
NJ = pd.read_excel("Data/Pre1790/liquidated_debt_certificates_NJ.xlsx", header = 10)
cols = ["JPEG number","Number","Letter","Date of the Certificate: Month",
        "Date of the Certificate: Day","Date of the Certificate: Year","Title","First name","Last name",
        "Title 2","First name 2","Last name 2","Time when the debt became due: Month",
        "Time when the debt became due: Day", "Time when the debt became due: Year","Dollars","90th",
        "Strike Through Number","Note"]
NJ = NJ[NJ.columns[0:19]]
NJ.columns = cols

NJ = NJ[NJ['First name'].apply(lambda x: not pd.isnull(x))]

#create full name
NJ['full name 1'] = NJ['First name'] + " " + NJ['Last name']
NJ['full name 2'] = NJ['First name 2'] + " " + NJ['Last name 2']

In [261]:
NJ_short = NJ[['full name 1', 'full name 2', 'Number', 'Dollars', '90th']]
NJ_short = NJ_short[NJ_short['Number'].apply(lambda x: not pd.isnull(x))]
NJ_short['Number'] = NJ_short['Number'].apply(lambda x: 'NJ_' + str(int(x)))
states_short = pd.concat([states_short, NJ_short])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


# New York

In [262]:
#Read in file
NY = pd.read_excel("Data/Pre1790/liquidated_debt_certificates_NY.xlsx", header = 10)
cols = ["Page", "JPEG number","Number","Letter","Date of the Certificate: Month",
        "Date of the Certificate: Day","Date of the Certificate: Year","First name","Last name",
        "Title","First name 2","Last name 2","Title 2","Time when the debt became due: Month",
        "Time when the debt became due: Day", "Time when the debt became due: Year","Dollars","90th",
        "Strike Through Yes?","Note", "Notes"]
NY = NY[NY.columns[0:21]]
NY.columns = cols

NY = NY[NY['First name'].apply(lambda x: not pd.isnull(x))]

#create full name
NY['full name 1'] = NY['First name'] + " " + NY['Last name']
NY['full name 2'] = NY['First name 2'] + " " + NY['Last name 2']

In [263]:
NY_short = NY[['full name 1', 'full name 2', 'Number', 'Dollars', '90th']]
NY_short = NY_short[NY_short['Number'].apply(lambda x: not pd.isnull(x))]
NY_short['Number'] = NY_short['Number'].apply(lambda x: 'NY_' + str(int(x)))
states_short = pd.concat([states_short, NY_short])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


# Pennsylvania P1

In [264]:
#Read in file
PA = pd.read_excel("Data/Pre1790/liquidated_debt_certificates_PA_story.xlsx", header = 11)
cols = ["JPEG number","Number","Letter","Date of the Certificate: Year",
        "Date of the Certificate: Month","Date of the Certificate: Date","Title","First name","Last name","Time when the debt became due: Year",
        "Time when the debt became due: Month","Time when the debt became due: Day",
        "Dollars","90th","10th", "Exchange","Amount in Specie_1", "Amount in Specie_2","Line Strike Thorugh?","Notes"]
PA = PA[PA.columns[1:21]]
PA.columns = cols

PA = PA[PA['First name'].apply(lambda x: not pd.isnull(x))]

#create full name
PA['full name 1'] = PA['First name'] + " " + PA['Last name']

In [265]:
PA_short_1 = PA[['full name 1', 'Number', 'Dollars', '90th']]
PA_short_1 = PA_short_1[PA_short_1['Number'].apply(lambda x: not pd.isnull(x))]
PA_short_1['Number'] = PA_short_1['Number'].apply(lambda x: 'PA_1_' + str(int(x)))
states_short = pd.concat([states_short, PA_short_1])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


# Pennsylvania P2

In [266]:
#Read in file
PA2 = pd.read_excel("Data/Pre1790/liquidated_debt_certificates_PA_stelle.xlsx", header = 11)
cols = ["JPEG number","Number","Letter","Date of the Certificate: Year",
        "Date of the Certificate: Month","Date of the Certificate: Date","Title","First name","Last name","Title 2","First name 2","Last name 2",
        "Time when the debt became due: Year", "Time when the debt became due: Month","Time when the debt became due: Day",
        "Dollars","90th","Line Strike Thorugh?","Notes"]
PA2 = PA2[PA2.columns[1:21]]
PA2.columns = cols

PA2 = PA2[PA2['First name'].apply(lambda x: not pd.isnull(x))]

#create full name
PA2['full name 1'] = PA2['First name'] + " " + PA2['Last name']
PA2['full name 2'] = PA2['First name 2'] + " " + PA2['Last name 2']

In [267]:
PA_short_2 = PA2[['full name 1', 'full name 2', 'Number', 'Dollars', '90th']]
PA_short_2 = PA_short_2[PA_short_2['Number'].apply(lambda x: not pd.isnull(x))]
PA_short_2['Number'] = PA_short_2['Number'].apply(lambda x: 'PA_2_' + str(int(x)))
states_short = pd.concat([states_short, PA_short_2])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


# Rhode Island

In [268]:
#Read in file
RI = pd.read_excel("Data/Pre1790/liquidated_debt_certificates_RI.xlsx", header = 12)
cols =  ["Page", "JPEG number","Number","Letter","Date of the Certificate: Month",
        "Date of the Certificate: Day","Date of the Certificate: Year","First name","Last name",
        "Title","First name 2","Last name 2","Title 2","Time when the debt became due: Month",
        "Time when the debt became due: Day", "Time when the debt became due: Year","Dollars","90th",
        "8th"]
RI = RI[RI.columns[0:19]]
RI.columns = cols

RI = RI[RI['First name'].apply(lambda x: not pd.isnull(x))]

#create full name
RI['full name 1'] = RI['First name'] + " " + RI['Last name']
RI['full name 2'] = RI['First name 2'] + " " + RI['Last name 2']

In [269]:
RI_short = RI[['full name 1', 'full name 2', 'Number', 'Dollars', '90th']]
RI_short = RI_short[RI_short['Number'].apply(lambda x: not pd.isnull(x))]
RI_short['Number'] = RI_short['Number'].apply(lambda x: 'RI_' + str(int(x)))
states_short = pd.concat([states_short, RI_short])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


# Final Steps

In [270]:
state_cert_assets = pd.merge(total_assets_final,
                                   states_short[states_short['full name 1'].apply(lambda x: not pd.isnull(x))],
                                   left_on = 'match name', right_on = 'full name 1').drop('full name 1', axis = 1)
add_to_assets = pd.merge(total_assets_final,
                         states_short[states_short['full name 2'].apply(lambda x: not pd.isnull(x))],
                         left_on = 'match name', right_on = 'full name 2').drop('full name 2', axis = 1)
state_cert_assets = pd.concat([state_cert_assets, add_to_assets])
state_cert_assets = state_cert_assets.drop_duplicates(['delegate name', 'Number'])
state_cert_assets['delegate_state'] = state_cert_assets['delegate_state'].apply(lambda x: 'Constitution' if pd.isnull(x) else x)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  import sys


In [271]:
state_cert_assets.rename(columns={'Dollars':'State Certificates Dollars','90th':'State Certificates 90th'}, inplace = True)

In [272]:
state_cert_assets = pd.merge(cum_firstlast, state_cert_assets, left_on = 'Full Name', right_on = 'delegate name').drop('Full Name', axis = 1).drop_duplicates()
state_cert_assets_agg = state_cert_assets.groupby(['delegate name', 'First Name', 'Last Name',  'delegate_state', 'sign'])[['State Certificates Dollars', 'State Certificates 90th']].agg(['sum','count'])
state_cert_assets_agg

State Certificates Dollars  \
                                                                                   sum   
delegate name      First Name Last Name delegate_state sign                              
Abraham Smith      Abraham    Smith     MA             no                          4.0   
Abraham Stout      Abraham    Stout     PA             yes                        14.0   
Albert Baker       Albert     Baker     NY             no                         28.0   
Alexander Hamilton Alexander  Hamilton  Constitution   yes                       160.0   
Alexander Martin   Alexander  Martin    Constitution   no                        124.0   
...                                                                                ...   
William White      William    White     VA             no                        721.0   
William Williams   William    Williams  CT             yes                      3552.0   
William Wilson     William    Wilson    PA             yes                        96.0   
Williams John      Williams   John      RI             no                       1766.0   
Zephaniah Platt    Zephaniah  Platt     NY             yes                       284.0   

                                                                   \
                                                            count   
delegate name      First Name Last Name delegate_state sign         
Abraham Smith      Abraham    Smith     MA             no       1   
Abraham Stout      Abraham    Stout     PA             yes      2   
Albert Baker       Albert     Baker     NY             no       1   
Alexander Hamilton Alexander  Hamilton  Constitution   yes      1   
Alexander Martin   Alexander  Martin    Constitution   no       2   
...                                                           ...   
William White      William    White     VA             no      12   
William Williams   William    Williams  CT             yes     14   
William Wilson     William    Wilson    PA             yes      3   
Williams John      Williams   John      RI             no      19   
Zephaniah Platt    Zephaniah  Platt     NY             yes      1   

                                                            State Certificates 90th  \
                                                                                sum   
delegate name      First Name Last Name delegate_state sign                           
Abraham Smith      Abraham    Smith     MA             no                         0   
Abraham Stout      Abraham    Stout     PA             yes                      136   
Albert Baker       Albert     Baker     NY             no                         0   
Alexander Hamilton Alexander  Hamilton  Constitution   yes                        0   
Alexander Martin   Alexander  Martin    Constitution   no                       110   
...                                                                             ...   
William White      William    White     VA             no                       383   
William Williams   William    Williams  CT             yes                      486   
William Wilson     William    Wilson    PA             yes                        6   
Williams John      Williams   John      RI             no                       579   
Zephaniah Platt    Zephaniah  Platt     NY             yes                       26   

                                                                   
                                                            count  
delegate name      First Name Last Name delegate_state sign        
Abraham Smith      Abraham    Smith     MA             no       0  
Abraham Stout      Abraham    Stout     PA             yes      2  
Albert Baker       Albert     Baker     NY             no       0  
Alexander Hamilton Alexander  Hamilton  Constitution   yes      0  
Alexander Martin   Alexander  Martin    Constitution   no       2  
...                                                           ...  
W

In [273]:
loan_state_merge = pd.merge(loan_office_cert_assets_agg, state_cert_assets_agg, on = ["delegate name", "First Name", "Last Name", "delegate_state", "sign"],how = "outer")

In [274]:
loan_state_marine_merge = pd.merge(loan_state_merge, marine_cert_assets_agg, on = ["delegate name", "First Name", "Last Name", "delegate_state", "sign"],how = "outer")

In [275]:
loan_state_marine_pierce_merge = pd.merge(loan_state_marine_merge, pierce_cert_assets_agg, on = ["delegate name", "First Name", "Last Name", "delegate_state", "sign"],how = "outer")

In [276]:
loan_state_marine_pierce_merge

Loan Office Certificates Face Value  \
                                                                                             sum   
delegate name      First Name Last Name  delegate_state sign                                       
Aaron Austin       Aaron      Austin     CT             yes                               3700.0   
Aaron Wood         Aaron      Wood       MA             no                                 600.0   
Abraham Faw        Abraham    Faw        MD             yes                                400.0   
Abraham Holmes     Abraham    Holmes     MA             no                               13200.0   
Abraham Smith      Abraham    Smith      MA             no                                 300.0   
...                                                                                          ...   
William Pinkney    William    Pinkney    MD             no                                   NaN   
William R. Davie   William R. Davie      Constitution   no                                   NaN   
William Thornton   William    Thornton   VA             yes                                  NaN   
William Washington William    Washington SC             yes                                  NaN   
Zachariah Johnston Zachariah  Johnston   VA             yes                                  NaN   

                                                                    \
                                                             count   
delegate name      First Name Last Name  delegate_state sign         
Aaron Austin       Aaron      Austin     CT             yes    3.0   
Aaron Wood         Aaron      Wood       MA             no     1.0   
Abraham Faw        Abraham    Faw        MD             yes    1.0   
Abraham Holmes     Abraham    Holmes     MA             no     8.0   
Abraham Smith      Abraham    Smith      MA             no     1.0   
...                                                            ...   
William Pinkney    William    Pinkney    MD             no     NaN   
William R. Davie   William R. Davie      Constitution   no     NaN   
William Thornton   William    Thornton   VA             yes    NaN   
William Washington William    Washington SC             yes    NaN   
Zachariah Johnston Zachariah  Johnston   VA             yes    NaN   

                                                             Loan Office Certificates Specie Value  \
                                                                                               sum   
delegate name      First Name Last Name  delegate_state sign                                         
Aaron Austin       Aaron      Austin     CT             yes                              180.06667   
Aaron Wood         Aaron      Wood       MA             no                                66.72500   
Abraham Faw        Abraham    Faw        MD             yes                               35.53889   
Abraham Holmes     Abraham    Holmes     MA             no                               527.52500   
Abraham Smith      Abraham    Smith      MA             no                               123.49170   
...                                                                                            ...   
William Pinkney    William    Pinkney    MD             no                                     NaN   
William R. Davie   William R. Davie      Constitution   no                                     NaN   
William Thornton   William    Thornton   VA             yes                                    NaN   
William Washington William    Washington SC             yes                                    NaN   
Zachariah Johnston Zachariah  Johnston   VA             yes                                    NaN   

                                                                    \
                                                             count   
delegate name      First Name Last Name  delegate_state sign         
Aaron Austin       Aaron      Austin    

In [277]:
cum_delegate_list = pd.merge(total_assets, cum_firstlast, left_on = 'delegate name', right_on = 'Full Name').drop_duplicates().drop(['match name', 'Full Name'], axis = 1)
spec_index = cum_delegate_list[cum_delegate_list['delegate_state'].apply(lambda x: pd.isnull(x))].index
cum_delegate_list.loc[spec_index, ['delegate_state']] = 'Constitution'
cum_delegate_list = cum_delegate_list[['delegate name', 'First Name', 'Last Name', 'delegate_state', 'sign']]
for row in cum_delegate_list.index:
    row_vals = tuple(cum_delegate_list.loc[row].tolist())
    if row_vals not in loan_state_marine_pierce_merge.index:
        loan_state_marine_pierce_merge.ix[row_vals,:] = [np.nan] * 14

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  


In [282]:
loan_state_marine_pierce_merge = loan_state_marine_pierce_merge.sort_values('Last Name')

In [284]:
loan_office_cert_assets_agg.to_csv('Producables/Separate/Loan_Office_Certificates_Matched.csv')
pierce_cert_assets_agg.to_csv('Producables/Separate/Pierce_Certificates_Matched.csv')
marine_cert_assets_agg.to_csv('Producables/Separate/Marine_Certificates_Matched.csv')
state_cert_assets_agg.to_csv('Producables/Separate/State_Certificates_Matched.csv')

In [285]:
loan_state_marine_pierce_merge.to_csv('Producables/Delegate_Pre1790_Assets.csv')